In [3]:
import pandas as pd
import glob
import numpy as np

# Helper Functions

In [144]:
def load_fer_output_data(path):
    csv_files = glob.glob(path + "/*.csv")
    csv_files.sort()

    df_list = (pd.read_csv(file) for file in csv_files)
 
    # Concatenate all DataFrames
    df = pd.concat(df_list, ignore_index=True)
    df.columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
    df['start']= [0.1*count for count in range(len(df))]
    df['end'] = [0.1*count for count in range(1,len(df)+1)]

    return df

In [6]:
def load_patch_fer_output_data(path):
    csv_files = glob.glob(path + "/*")
    csv_files.sort()

    df_list = [pd.read_csv(file).mean().to_list() for file in csv_files]
    
    df = pd.DataFrame(df_list, columns=['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'])
    df['start_patch']= [5*count for count in range(len(df))]
    # df['end'] = [5*count for count in range(1,len(df)+1)]


    return df

In [181]:
def merge_text_fer_data(df_facial, df_text):
    df_text = df_text.copy()
    df_facial = df_facial.copy()
    df_facial = df_facial.drop('end', axis=1)
    df_facial.columns = ['facial_anger', 'facial_disgust', 'facial_fear', 'facial_joy', 'facial_sadness', 'facial_surprise', 'facial_neutral', 'start_patch']
    df_text['start_patch'] = np.floor(((df_text['start']/60)/5))*5
    df_text['start'] = np.floor((df_text['start']/60))
    df_text = df_text.groupby('start_patch')['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'].mean().reset_index()
    df_text.columns = ['start_patch', 'text_anger', 'text_disgust', 'text_fear', 'text_joy', 'text_sadness', 'text_surprise', 'text_neutral']
    df_merged = pd.merge(df_text, df_facial, on=['start_patch'], how='inner').drop('start_patch', axis=1)
    return df_merged

In [228]:
def load_aggregate_hume_emotions(path):
    # Emotion mapping 
    ANGER = ['Anger', 'Contempt', 'Determination', 'Envy']
    DISGUST = ['Awkwardness', 'Disgust', 'Embarrassment', 'Tiredness']
    ENJOYMENT = ['Admiration','Adoration','Aesthetic Appreciation','Amusement','Contentment','Ecstasy','Entrancement','Joy','Love','Romance','Satisfaction','Sympathy','Triumph']
    FEAR = ['Anxiety','Awe','Distress','Doubt','Fear','Horror','Pain']
    NEUTRAL = ['Boredom','Calmness','Concentration','Contemplation']
    SADNESS = ['Confusion','Disappointment','Empathic Pain','Guilt','Nostalgia','Sadness','Shame']
    SURPRISE = ['Craving','Desire','Excitement','Interest','Pride','Realization','Relief','Surprise (negative)','Surprise (positive)']

    df_org = pd.read_csv(path, index_col=0)
    df = pd.DataFrame(df_org[ANGER].mean(axis=1), columns=['anger'])
    df['disgust'] = df_org[DISGUST].mean(axis=1)
    df['fear'] = df_org[FEAR].mean(axis=1)
    df['joy'] = df_org[ENJOYMENT].mean(axis=1)
    df['sadness'] = df_org[SADNESS].mean(axis=1)
    df['surprise'] = df_org[SURPRISE].mean(axis=1)
    df['neutral'] = df_org[NEUTRAL].mean(axis=1)

    df['start_patch'] = np.floor(((df_org['time']/60)/5))*5
    # df = df.groupby('start_patch')['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'].mean().reset_index()
    
    return df


# FER/Text Correlations

In [52]:
df_facial_vocals_morning = load_patch_fer_output_data('../data/FER_Output/2_jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp/2_csv')
df_vocal_afternoon = load_patch_fer_output_data('../data/FER_Output/1_jazzaar-afternoon-JFLB-Cuban-Vocals-video/1_csv')
df_rhythm_morning = load_patch_fer_output_data('../data/FER_Output/6_5-JFBB_Rythm_Morning/6_csv')
df_rhythm_afternoon = load_patch_fer_output_data('../data/FER_Output/5-JFBB_Rythm_Afternoon/5_csv')
df_band_morning = load_patch_fer_output_data('../data/FER_Output/4_JFBB_Jazz_Funk_morning/4_csv')
df_band_afternoon = load_patch_fer_output_data('../data/FER_Output/3_JFBB_Jazz_Funk_afternoon_video20230419143359 Luis Cal García/3_csv')


In [54]:
df_facial_vocals_morning = df_facial_vocals_morning.to_csv('../data/FER_Output/processed/Cuban_vocals_morning.csv')
df_vocal_afternoon = df_vocal_afternoon.to_csv('../data/FER_Output/Cuban_vocals_afternoon.csv')
df_rhythm_morning = df_rhythm_morning.to_csv('../data/FER_Output/Funk_rhythm_morning.csv')
df_rhythm_afternoon = df_rhythm_afternoon.to_csv('../data/FER_Output/Funk_rhythm_afternoon.csv')
df_band_morning = df_band_morning.to_csv('../data/FER_Output/Funk_band_morning.csv')
df_band_afternoon = df_band_afternoon.to_csv('../data/FER_Output/Funk_band_afternoon.csv')


In [ ]:
df_rhythm_morning.to_csv('../data/hume/processed/Funk_rhythm_morning.csv')
df_band_morning.to_csv('../data/hume/processed/Funk_band_morning.csv')
df_vocals_afternoon.to_csv('../data/hume/processed/Cuban_vocals_afternoon.csv')
df_vocals_morning.to_csv('../data/hume/processed/Cuban_vocals_morning.csv')

In [185]:
df_text_vocals_morning = pd.read_csv('../data/emotion_from_transcript/emotion-english-distilroberta-base/jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp.csv', index_col=0)

In [207]:
df_vocals_morning_merged = merge_text_fer_data(df_facial_vocals_morning, df_text_vocals_morning)
df_vocals_morning_corr = df_vocals_morning_merged.corr().drop(['text_anger', 'text_disgust', 'text_fear', 'text_joy', 'text_sadness', 'text_surprise', 'text_neutral'], axis=1)
df_vocals_morning_corr = df_vocals_morning_corr.drop(['facial_anger', 'facial_disgust', 'facial_fear', 'facial_joy', 'facial_sadness', 'facial_surprise', 'facial_neutral'],axis=0)
df_vocals_morning_corr

/tmp/ipykernel_3620/4211503723.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_text = df_text.groupby('start_patch')['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'].mean().reset_index()


,facial_anger,facial_disgust,facial_fear,facial_joy,facial_sadness,facial_surprise,facial_neutral
text_anger,0.088131,-0.046638,-0.166281,-0.079025,0.032596,0.352811,-0.079247
text_disgust,-0.037573,-0.133239,-0.128602,0.218443,-0.005819,-0.116359,-0.048432
text_fear,0.136245,-0.338267,-0.332244,-0.136314,-0.360337,0.407742,0.314530
text_joy,0.036551,0.213681,0.436095,-0.045756,0.077573,-0.260738,-0.201377
text_sadness,-0.227920,0.429767,-0.056182,0.335701,0.168237,-0.068511,-0.160812
text_surprise,0.510384,-0.044646,0.179982,-0.226090,-0.188372,-0.008098,-0.266579
text_neutral,-0.286219,-0.221322,-0.272066,-0.028241,0.021881,0.135383,0.409433


In [219]:
df_facial_vocals_morning = df_facial_vocals_morning.drop('end', axis=1)
df_facial_vocals_morning.columns = ['facial_anger', 'facial_disgust', 'facial_fear', 'facial_joy', 'facial_sadness', 'facial_surprise', 'facial_neutral', 'start_patch']
df_vocals_morning_aggregated=pd.DataFrame(df_facial_vocals_morning[['facial_anger', 'facial_disgust','facial_fear', 'facial_sadness']].mean(axis=1), columns=['facial_agg_negative'])
df_vocals_morning_aggregated['facial_agg_positive'] = df_facial_vocals_morning[['facial_neutral', 'facial_surprise','facial_joy']].mean(axis=1)
df_vocals_morning_aggregated['text_agg_positive'] = df_text_vocals_morning[['text_neutral', 'text_surprise','text_joy']].mean(axis=1)
df_vocals_morning_aggregated['text_agg_negative'] = df_text_vocals_morning[['text_anger', 'text_disgust','text_fear', 'text_sadness']].mean(axis=1)
df_vocals_morning_corr = df_vocals_morning_aggregated.corr().drop(['text_agg_positive', 'text_agg_negative'], axis=1)
df_vocals_morning_corr = df_vocals_morning_corr.drop(['facial_agg_positive', 'facial_agg_negative'], axis=0)
df_vocals_morning_corr

KeyError: "['end'] not found in axis"

# Hume/Text correlations

In [230]:
df_facial_vocals_morning_hume = load_aggregate_hume_emotions('../data/hume/raw_predictions/jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp.csv')
max(df_facial_vocals_morning_hume['start_patch'])

0.0

In [232]:
def load_aggregate_hume_emotions(path):
    # Emotion mapping 
    ANGER = ['Anger', 'Contempt', 'Determination', 'Envy']
    DISGUST = ['Awkwardness', 'Disgust', 'Embarrassment', 'Tiredness']
    ENJOYMENT = ['Admiration','Adoration','Aesthetic Appreciation','Amusement','Contentment','Ecstasy','Entrancement','Joy','Love','Romance','Satisfaction','Sympathy','Triumph']
    FEAR = ['Anxiety','Awe','Distress','Doubt','Fear','Horror','Pain']
    NEUTRAL = ['Boredom','Calmness','Concentration','Contemplation']
    SADNESS = ['Confusion','Disappointment','Empathic Pain','Guilt','Nostalgia','Sadness','Shame']
    SURPRISE = ['Craving','Desire','Excitement','Interest','Pride','Realization','Relief','Surprise (negative)','Surprise (positive)']

    df_org = pd.read_csv(path, index_col=0)
    df = pd.DataFrame(df_org[ANGER].mean(axis=1), columns=['anger'])
    df['disgust'] = df_org[DISGUST].mean(axis=1)
    df['fear'] = df_org[FEAR].mean(axis=1)
    df['joy'] = df_org[ENJOYMENT].mean(axis=1)
    df['sadness'] = df_org[SADNESS].mean(axis=1)
    df['surprise'] = df_org[SURPRISE].mean(axis=1)
    df['neutral'] = df_org[NEUTRAL].mean(axis=1)

    df['start_patch'] = np.floor(((df_org['time']/60)/5))*5
    # df = df.groupby('start_patch')['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'].mean().reset_index()
    
    return df


In [198]:
df_facial_vocals_morning_hume.head()

AttributeError: 'NoneType' object has no attribute 'head'